In [8]:
from read_data import getData
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from read_data import LargeDataset
import gc




In [9]:
traindata, testData = getData("data")
traindata.shape, testData.shape

Training Data's shape is (10000, 50, 110, 6) and Test Data's is (10000, 50, 110, 6)


((10000, 50, 110, 6), (2100, 50, 50, 6))

In [10]:
train_mean = np.mean(traindata, axis=(0, 1, 2))
train_std = np.std(traindata, axis=(0, 1, 2))
train_std = np.where(train_std == 0, 1.0, train_std)

In [12]:
class WindowedNormalizedDataset(Dataset):
    def __init__(self, data, window_size=40, forecast_horizon=10, mean=None, std=None):
        self.data = data
        self.window_size = window_size
        self.forecast_horizon = forecast_horizon
        self.mean = mean
        self.std = std

        # Precompute indices of valid (sample, t) combinations
        self.indices = []
        for sample in range(data.shape[0]):
            for t in range(data.shape[2] - window_size - forecast_horizon + 1):
                self.indices.append((sample, t))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        sample_idx, t = self.indices[idx]
        
        x = self.data[sample_idx, :, t:t+self.window_size, :]  # shape: (50, 40, 6)
        y = self.data[sample_idx, 0, t+self.window_size:t+self.window_size+self.forecast_horizon, :2]  # shape: (10, 2)

        if self.mean is not None and self.std is not None:
            x = (x - self.mean) / self.std
        
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


dataset = WindowedNormalizedDataset(traindata)


In [13]:
# def createDataset(data, window_size = 40, forecast_horizon = 10):
#     X = []
#     y = []

#     for sample in range(data.shape[0]):
#         for t in range(data.shape[2] - window_size - forecast_horizon + 1):
#             x_window = data[sample, :, t:t+window_size, :]
#             y_window = data[sample, 0, t+window_size:t+window_size+forecast_horizon, :2]
            
#             X.append(x_window)
#             y.append(y_window)
    
#     return np.array(X), np.array(y)


# X, Y = createDataset(traindata)
# X.shape, Y.shape

In [ ]:
# a, b = X[:10000], Y[:10000]
# a.shape, b.shape

((10000, 50, 40, 6), (10000, 10, 2))

In [14]:
device = torch.device("mps")

class SmallNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size = 6, hidden_size = 64, num_layers = 5, batch_first = True)
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.pool = nn.AdaptiveAvgPool1d(10)
        self.linear1 = nn.Linear(64, 32)
        self.linear2 = nn.Linear(32, 16)
        self.linear3 = nn.Linear(16, 2)

    def forward(self, x):
        x = x.view(x.size(0), -1, x.size(-1))

        x, temp = self.lstm(x) # Output shape [batch, seq_len, 64]
        x = x.permute(0, 2, 1)  # [batch, 64, seq_len]
       

        x = self.pool(x)  # Forces output to [batch, 64, 10]
        x = self.batch_norm1(x)
        x = x.permute(0, 2, 1)  # [batch, 10, 64]
        x = self.linear1(x)
        x = torch.nn.functional.leaky_relu(x, negative_slope=0.01)
        x = self.linear2(x)
        x = torch.nn.functional.leaky_relu(x, negative_slope=0.01)
        x = self.linear3(x)
        return x

model = SmallNetwork()
# model.to(device)

# test = torch.randn(2, 2, 2, 6)
# out = model(test)
# print(out.shape)

In [18]:
# trainDataset = LargeDataset(a, b, train_mean, train_std) # testing for small dataset a, b
trainDataLoader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)

model.to(device)

# Training setup
epochs = 100
lossFn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for each_epoch in range(epochs):
    model.train()
    runningLoss = 0.0
    loop = tqdm(trainDataLoader, desc=f"Epoch [{each_epoch+1}/{epochs}]")

    for batchX, batchY in loop:
        batchX, batchY = batchX.to(device, non_blocking=True), batchY.to(device, non_blocking=True)
        output = model(batchX)
        loss = lossFn(output, batchY)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()

    avgLoss = runningLoss / len(trainDataLoader)

    if each_epoch % 5 == 0:
        torch.save(model.state_dict(), f'./models/small_model_{each_epoch}.pth')
        print(f"Epoch {each_epoch + 1}, Training Loss: {avgLoss:.4f}")


Epoch [1/100]:   0%|          | 0/9532 [00:00<?, ?it/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
    from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=79, pipe_handle=117)
                                                  ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ishananand/miniforge3/envs/pytorch_env/lib/python3.13/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/ishananand/miniforge3/envs/pytorch_env/lib/python3.13/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'WindowedNormalizedDataset' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
Epoch [1/100]:   0%|          | 0/9532 [1:05:54<?, ?it/s]


KeyboardInterrupt: 